In [1]:
import torch as t
import torch.nn as nn
import torch.optim as optim
import torchvision._utils as util
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import seaborn as sns
import numpy as np
import pandas as pd
import cv2 as cv
import os
import sklearn.model_selection as sc
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F


In [2]:
folder_path = "/Users/bhara-zstch1566/CNN/Project/Resnet real/archive-3/brain_tumor_dataset/no"

X = []
Y = []
print(len(os.listdir(folder_path)))

for foldername in os.listdir(folder_path):
    
    img = (cv.imread("/Users/bhara-zstch1566/CNN/Project/Resnet real/archive-3/brain_tumor_dataset/no/"+foldername))    
    img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    X.append(img)
    Y.append(0)

X_test = X[90:]
X = X[:90]

Y_test = Y[90:]
Y = Y[:90]

currx=[]
curry =[]
folder_path = "/Users/bhara-zstch1566/CNN/Project/Resnet real/archive-3/brain_tumor_dataset/yes"

for foldername in os.listdir(folder_path):
    img = (cv.imread("/Users/bhara-zstch1566/CNN/Project/Resnet real/archive-3/brain_tumor_dataset/yes/"+foldername))
    img = cv.cvtColor(img, cv.COLOR_BGR2RGB)

    currx.append(img)
    curry.append(1)

X_test += currx[90:]
currx = currx[:90]

Y_test += curry[90:]
curry = curry[:90]

X += currx
Y += curry

print(len(os.listdir(folder_path)))


FileNotFoundError: [Errno 2] No such file or directory: '/Users/bhara-zstch1566/CNN/Project/Resnet real/archive-3/brain_tumor_dataset/no'

In [ ]:
print(len(X),len(Y))

180 180


In [ ]:
transform = transforms.Compose([
    transforms.ToPILImage(),                   
    transforms.Grayscale(num_output_channels=1),  # convert to grayscale
    transforms.Resize((64, 64)),                  # resize
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])   # normalize grayscale
])
def ta(x):
    for i in range(len(x)):
        x[i] = transform(x[i])
    return x
X = ta(X)

In [ ]:
X = t.stack(X)

In [ ]:
X.shape

torch.Size([180, 1, 64, 64])

In [ ]:
Y= t.tensor(Y)

In [ ]:
print(X.shape,Y.shape)

torch.Size([180, 1, 64, 64]) torch.Size([180])


In [3]:
class BrainTumor(nn.Module):
    def __init__(self):
        super(BrainTumor,self).__init__()
        self.conv1 = nn.Conv2d(1,3,5,1)
        self.conv2 = nn.Conv2d(3,256,5,2)
        self.max_pool = nn.MaxPool2d(3,2)
        self.conv3 = nn.Conv2d(256,256,3)
        self.net = nn.Sequential(
            nn.Linear(4096,4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096,4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096,2)
        )
    
    def forward(self,x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.max_pool(x)
        x = F.relu(self.conv3(F.relu(self.conv3(x))))
        x = self.max_pool(x)
        x = x.view(x.size(0), -1)
        x = self.net(x)
        return x

In [4]:
model = BrainTumor()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [5]:
train_dataset = TensorDataset(X,Y)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

AttributeError: 'list' object has no attribute 'size'

In [12]:
device = "mps" if t.mps.is_available() else "cpu"
model = model.to(device)

In [13]:
for epoch in range(5):
    running = 0.0
    for img,label in train_loader:
        img , label = img.to(device),label.to(device)
        y_pred = model(img)
        loss = criterion(y_pred,label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running +=loss.item()
    
    print(f"Epoch [{epoch+1}/{5}], Loss: {running / len(train_loader):.4f}")


Epoch [1/5], Loss: 0.7360
Epoch [2/5], Loss: 0.6963
Epoch [3/5], Loss: 0.6642
Epoch [4/5], Loss: 0.7245
Epoch [5/5], Loss: 0.6469


In [17]:
Y_test = t.tensor(Y_test)
X_test = ta(X_test)
X_test = t.stack(X_test)

/var/folders/zm/gf8ny5z13m70t3shtcspf3rh0000gp/T/ipykernel_38358/4044436238.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Y_test = t.tensor(Y_test)


In [18]:
test_dataset = TensorDataset(X_test,Y_test)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)


In [19]:
with t.no_grad():
    ttotal =0
    tcet = 0
    for img,label in test_loader:
        img ,label = img.to(device),label.to(device)
        y_pred = model(img)
        _,predicted = t.max(y_pred.data, 1)
        ttotal += label.size(0)
        tcet += (predicted == label).sum().item()
    print(f"Accuracy Accuracy: {100*tcet/ttotal:.2f}%")

Accuracy Accuracy: 79.45%
